In [1]:
import pandas as pd
import os

In [95]:
economics = pd.read_csv(r'SAINC_2000_2020.csv')
economics = economics.transpose()
economics = economics.drop(economics.index[0:4])

# headers = ['Year',
# 'Personal income (millions of dollars)',
# 'Nonfarm personal income 1',
# 'Farm income 2', 
# 'Population (persons) 3',
# 'Per capita personal income (dollars) 4',
# 'Earnings by place of work Less: Contributions for government social insurance 5',
# 'Employee and self-employed contributions for government social insurance',
# 'Employer contributions for government social insurance	Plus: Adjustment for residence',
# 'Proprietors income', 
# 'Farm proprietors income',
# 'Nonfarm proprietors income',	
# 'Total employment',	
# 'Wage and salary employment',	
# 'Proprietors employment',	
# 'NaN',	
# 'NaN',	
# 'NaN',	
# 'NaN']

# economics.columns = economics.columns.tolist() + headers


# headers = economics.iloc[4]
# economics = pd.DataFrame(economics.values[1:], columns=headers)
# economics = economics.rename(columns=economics.iloc[0])

In [96]:
economics.head()


,0,1,2,3,4,5,6,7,8,9,...,1374,1375,1376,1377,1378,1379,1380,1381,1382,1383
Description,Personal income (millions of dollars),Nonfarm personal income 1/,Farm income 2/,Population (persons) 3/,Per capita personal income (dollars) 4/,Earnings by place of work,Less: Contributions for government social ins...,Employee and self-employed contributions for...,Employer contributions for government social...,Plus: Adjustment for residence 6/,...,Proprietors' income 9/,Farm proprietors' income,Nonfarm proprietors' income,Total employment,Wage and salary employment,Proprietors employment,NaN,NaN,NaN,NaN
Unit,Millions of dollars,Millions of dollars,Millions of dollars,Number of persons,Dollars,Millions of dollars,Millions of dollars,Millions of dollars,Millions of dollars,Millions of dollars,...,Millions of dollars,Millions of dollars,Millions of dollars,Number of jobs,Number of jobs,Number of jobs,NaN,NaN,NaN,NaN
2000,8654561.0,8603301.0,51260.0,282162411.0,30672.0,6598743.0,704849.0,360123.0,344726.0,2587.0,...,156005.2,4349.2,151656.0,27241357.0,22203631.0,5037726.0,NaN,NaN,NaN,NaN
2001,9009842.0,8956153.0,53689.0,284968955.0,31617.0,6865291.0,732117.0,374669.0,357448.0,2692.0,...,165750.1,3778.3,161971.8,27368668.0,22309853.0,5058815.0,NaN,NaN,NaN,NaN
2002,9157682.0,9115873.0,41809.0,287625193.0,31839.0,7001002.0,750339.0,384873.0,365466.0,2657.0,...,178486.0,4775.2,173710.9,27368961.0,22190073.0,5178888.0,NaN,NaN,NaN,NaN


In [83]:
inflation = pd.read_csv(r'InflationTable.csv')
inflation.head()

,Year,Month,Inflation_Factor
0,2000,Jan,0.645304
1,2001,Jan,0.669389
2,2002,Jan,0.677034
3,2003,Jan,0.694620
4,2004,Jan,0.708000


In [ ]:
df = pd.merge(economics, inflation, how='left', on ='')